## Lab 7.04 | Lab | Random Forests

Importing databases and concatenating all 3 datasets 

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
targets = pd.read_csv('target.csv')
data = pd.concat([numerical, categorical, targets], axis = 1) #concat because we can treat for imbalance, we use label to select which group certain labels belong to
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [2]:
data.head()

TCODE        AGE  INCOME  WEALTH1  HIT  MALEMILI  MALEVET  VIETVETS  \
0      0  60.000000       5        9    0         0       39        34   
1      1  46.000000       6        9   16         0       15        55   
2      1  61.611649       3        1    2         0       20        29   
3      0  70.000000       1        4    2         0       23        14   
4      0  78.000000       3        2   60         1       28         9   

   WWIIVETS  LOCALGOV  STATEGOV  FEDGOV  WEALTH2  POP901  POP902  POP903  \
0        18        10         2       1        5     992     264     332   
1        11         6         2       1        9    3611     940     998   
2        33         6         8       1        1    7001    2040    2669   
3        31         3         0       3        0     640     160     219   
4        53        26         3       2        9    2520     627     761   

   POP90C1  POP90C2  POP90C3  POP90C4  POP90C5  ETH1  ETH2  ETH3  ETH4  ETH5  \
0        0       35       65       47       53    92     1     0     0    11   
1       99        0        0       50       50    67     0     0    31     6   
2        0        2       98       49       51    96     2     0     0     2   
3        0        8       92       54       46    61     0     0    11    32   
4       99        0        0       46       54     2    98     0     0     1   

   ETH6  ETH7  ETH8  ETH9  ETH10  ETH11  ETH12  ETH13  ETH14  ETH15  ETH16  \
0     0     0     0     0      0      0      0     11      0      0      0   
1     4     2     6     4     14      0      0      2      0      1      4   
2     0     0     0     0      0      0      0      2      0      0      0   
3     6     2     0     0      0      0      0     31      0      0      1   
4     0     0     0     0      0      0      0      0      0      0      0   

   AGE901  AGE902  AGE903  AGE904  AGE905  AGE906  AGE907  CHIL1  CHIL2  \
0      39      48      51      40      50      54      25     31     42   
1      34      41      43      32      42      45      32     33     46   
2      35      43      46      37      45      49      23     35     40   
3      32      40      44      34      43      47      25     45     35   
4      33      45      50      36      46      50      27     34     43   

   CHIL3  AGEC1  AGEC2  AGEC3  AGEC4  AGEC5  AGEC6  AGEC7  CHILC1  CHILC2  \
0     27     11     14     18     17     13     11     15      12      11   
1     21     13     14     33     23     10      4      2      11      16   
2     25     13     20     19     16     13     10      8      15      14   
3     20     15     25     17     17     12      7      7      20      17   
4     23     14     21     13     15     20     12      5      13      15   

   CHILC3  CHILC4  CHILC5  HHAGE1  HHAGE2  HHAGE3  HHN1  HHN2  HHN3  HHN4  \
0      34      25      18      26      10      23    18    33    49    28   
1      36      22      15      12       1       5     4    21    75    55   
2      30      22      19      25      10      23    21    35    44    22   
3      30      14      19      25      11      23    23    27    50    30   
4      34      19      19      31       7      27    16    26    57    36   

   HHN5  HHN6  MARR1  MARR2  MARR3  MARR4  HHP1  HHP2  DW1  DW2  DW3  DW4  \
0    12     4     61      7     12     19   198   276   97   95    2    2   
1    23     9     69      4      3     24   317   360   99   99    0    0   
2     6     2     63      9      9     19   183   254   69   69    1    6   
3    15     8     63      9      6     23   199   283   85   83    3    4   
4    24    14     42     17      9     33   235   323   99   98    0    0   

   DW5  DW6  DW7  DW8  DW9   HV1   HV2  HV3  HV4  HU1  HU2  HU3  HU4  HU5  \
0    0    0    7    7    0   479   635    3    2   86   14   96    4    7   
1    0    0    0    0    0  5468  5218   12   10   96    4   97    3    9   
2    5    3    3    3    0   497   546    2    1   78   22   93    7   18   
3  

Are there any NaN values?

In [3]:
data.isna().sum().sum()

0

In [4]:
data.columns

Index(['TCODE', 'AGE', 'INCOME', 'WEALTH1', 'HIT', 'MALEMILI', 'MALEVET',
       'VIETVETS', 'WWIIVETS', 'LOCALGOV',
       ...
       'MINRDATE_YR', 'MINRDATE_MM', 'MAXRDATE_YR', 'MAXRDATE_MM',
       'LASTDATE_YR', 'LASTDATE_MM', 'FIRSTDATE_YR', 'FIRSTDATE_MM',
       'TARGET_B', 'TARGET_D'],
      dtype='object', length=339)

In [5]:
numerical.shape

(95412, 315)

In [6]:
data.head()
data.shape

(95412, 339)

Dropping all of the categorical columns from yesterday's lab

In [7]:
data = data.drop(columns=['GEOCODE2','ODATEW_YR','ODATEW_MM','DOB_YR','DOB_MM','MINRDATE_YR','MINRDATE_MM','MAXRDATE_YR','MAXRDATE_MM','LASTDATE_YR','LASTDATE_MM','FIRSTDATE_YR','FIRSTDATE_MM'], axis = 0)

Dropping TARGET_D

In [9]:
data = data.drop(['TARGET_D'], axis=1).copy() #don't need it to do a classification
data.head()

TCODE        AGE  INCOME  WEALTH1  HIT  MALEMILI  MALEVET  VIETVETS  \
0      0  60.000000       5        9    0         0       39        34   
1      1  46.000000       6        9   16         0       15        55   
2      1  61.611649       3        1    2         0       20        29   
3      0  70.000000       1        4    2         0       23        14   
4      0  78.000000       3        2   60         1       28         9   

   WWIIVETS  LOCALGOV  STATEGOV  FEDGOV  WEALTH2  POP901  POP902  POP903  \
0        18        10         2       1        5     992     264     332   
1        11         6         2       1        9    3611     940     998   
2        33         6         8       1        1    7001    2040    2669   
3        31         3         0       3        0     640     160     219   
4        53        26         3       2        9    2520     627     761   

   POP90C1  POP90C2  POP90C3  POP90C4  POP90C5  ETH1  ETH2  ETH3  ETH4  ETH5  \
0        0       35       65       47       53    92     1     0     0    11   
1       99        0        0       50       50    67     0     0    31     6   
2        0        2       98       49       51    96     2     0     0     2   
3        0        8       92       54       46    61     0     0    11    32   
4       99        0        0       46       54     2    98     0     0     1   

   ETH6  ETH7  ETH8  ETH9  ETH10  ETH11  ETH12  ETH13  ETH14  ETH15  ETH16  \
0     0     0     0     0      0      0      0     11      0      0      0   
1     4     2     6     4     14      0      0      2      0      1      4   
2     0     0     0     0      0      0      0      2      0      0      0   
3     6     2     0     0      0      0      0     31      0      0      1   
4     0     0     0     0      0      0      0      0      0      0      0   

   AGE901  AGE902  AGE903  AGE904  AGE905  AGE906  AGE907  CHIL1  CHIL2  \
0      39      48      51      40      50      54      25     31     42   
1      34      41      43      32      42      45      32     33     46   
2      35      43      46      37      45      49      23     35     40   
3      32      40      44      34      43      47      25     45     35   
4      33      45      50      36      46      50      27     34     43   

   CHIL3  AGEC1  AGEC2  AGEC3  AGEC4  AGEC5  AGEC6  AGEC7  CHILC1  CHILC2  \
0     27     11     14     18     17     13     11     15      12      11   
1     21     13     14     33     23     10      4      2      11      16   
2     25     13     20     19     16     13     10      8      15      14   
3     20     15     25     17     17     12      7      7      20      17   
4     23     14     21     13     15     20     12      5      13      15   

   CHILC3  CHILC4  CHILC5  HHAGE1  HHAGE2  HHAGE3  HHN1  HHN2  HHN3  HHN4  \
0      34      25      18      26      10      23    18    33    49    28   
1      36      22      15      12       1       5     4    21    75    55   
2      30      22      19      25      10      23    21    35    44    22   
3      30      14      19      25      11      23    23    27    50    30   
4      34      19      19      31       7      27    16    26    57    36   

   HHN5  HHN6  MARR1  MARR2  MARR3  MARR4  HHP1  HHP2  DW1  DW2  DW3  DW4  \
0    12     4     61      7     12     19   198   276   97   95    2    2   
1    23     9     69      4      3     24   317   360   99   99    0    0   
2     6     2     63      9      9     19   183   254   69   69    1    6   
3    15     8     63      9      6     23   199   283   85   83    3    4   
4    24    14     42     17      9     33   235   323   99   98    0    0   

   DW5  DW6  DW7  DW8  DW9   HV1   HV2  HV3  HV4  HU1  HU2  HU3  HU4  HU5  \
0    0    0    7    7    0   479   635    3    2   86   14   96    4    7   
1    0    0    0    0    0  5468  5218   12   10   96    4   97    3    9   
2    5    3    3    3    0   497   546    2    1   78   22   93    7   18   
3  

#### Oversampling the data and creating 2 categories 

In [11]:
y = data['TARGET_B']
X= data.drop(['TARGET_B'], axis = 1)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [15]:
trainset = pd.concat ([X_train, y_train], axis=1)
trainset

TCODE        AGE  INCOME  WEALTH1  HIT  MALEMILI  MALEVET  VIETVETS  \
79401      1  75.000000       4        6    2         0       31        10   
86429      0  53.000000       5        9    0         0       29        24   
76729      1  60.000000       5        1    5         0       42        16   
38838      1  77.000000       6        6    9         1       40        23   
83012      3  55.000000       5        2   21        33       27        29   
...      ...        ...     ...      ...  ...       ...      ...       ...   
21243      0  70.000000       3        9    5         2       37        10   
45891      2  66.000000       3        6    0         0       36        11   
42613      1  61.611649       5        9    0         0       35        29   
43567      1  76.000000       5        3    1         0       59        22   
68268      0  61.611649       2        9    0         0       32        26   

       WWIIVETS  LOCALGOV  STATEGOV  FEDGOV  WEALTH2  POP901  POP902  POP903  \
79401        68         6        12       3        9     706     203     272   
86429        38         7         8       4        9    4526    1232    1592   
76729        62        10         2       1        1    6643    2027    3091   
38838        41         8         3       4        6    1388     397     486   
83012        18        12         2       7        9     991     268     389   
...         ...       ...       ...     ...      ...     ...     ...     ...   
21243        32        26         1       2        9    1158     354     397   
45891        62         4        16       0        9    1021     326     415   
42613        42        17         7       3        4    4492    1236    1827   
43567        31         3         3       0        3     794     224     282   
68268        37         3         6       2        9    3677    1100    1332   

       POP90C1  POP90C2  POP90C3  POP90C4  POP90C5  ETH1  ETH2  ETH3  ETH4  \
79401       99        0        0       47       53    75    21     0     2   
86429        0        0       99       49       51    75    16     9     0   
76729       99        0        0       45       55    98     0     0     1   
38838       99        0        0       49       51    78     1     1    15   
83012       93        0        7       50       50    88     8     0     3   
...        ...      ...      ...      ...      ...   ...   ...   ...   ...   
21243       99        0        0       50       50    98     0     0     1   
45891        0        0       99       50       50    95     5     0     0   
42613        0       71       29       49       51    97     0     3     0   
43567        0        0       99       47       53    76    23     0     0   
68268        9        0       91       50       50    98     0     1     0   

       ETH5  ETH6  ETH7  ETH8  ETH9  ETH10  ETH11  ETH12  ETH13  ETH14  ETH15  \
79401     2     0     0     0     0      0      1      0      0      1      0   
86429     1     0     0     0     0      0      0      0      1      0      0   
76729     2     0     0     0     0      0      0      0      0      1      0   
38838    11     0     1     5     6      1      1      1      7      1      0   
83012     4     0     0     0     1      1      0      0      2      1      0   
...     ...   ...   ...   ...   ...    ...    ...    ...    ...    ...    ...   
21243     4     0     0     1     0      0      0      0      2      0      0   
45891     0     0     0     0     0      0      0      0      0      0      0   
42613     1     0     0     0     0      0      0      0      1      0      0   
43567     4     0     0     0     0      0      0      0      3      1      0   
68268     1     0     0     0     0      0      0      0      0      0      0   

       ETH16  AGE901  AGE902  AGE903  AGE904  AGE905  AGE906  AGE907  CHIL1  \
79401      2      34      42      46      36      45      49      25     36   
86429      0      35      47      51     

In [16]:
from sklearn.utils import resample

category_0 = trainset[trainset['TARGET_B'] == 0]
category_1 = trainset[trainset['TARGET_B'] == 1]

In [17]:
category_1_oversampled = resample(category_1, 
                                  replace=True, #need to use replace, as you're taking it back and have the opportunity to put it back and take it away again
                                  n_samples = len(category_0))

In [18]:
print(category_0.shape)
print(category_1_oversampled.shape)

(72486, 325)
(72486, 325)


Concatenating the oversampled dataset

In [19]:
data_upsampled = pd.concat([category_0, category_1_oversampled], axis=0)

In [20]:
data_upsampled['TARGET_B'].value_counts()

0    72486
1    72486
Name: TARGET_B, dtype: int64

In [21]:
y = data_upsampled['TARGET_B']
X= data_upsampled.drop(['TARGET_B'], axis = 1)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Separating categorical and numerical train and test data and then one hot encoding first train then test set

In [40]:
X_train_num = X_train.select_dtypes(np.number).reset_index()
X_test_num = X_test.select_dtypes(np.number).reset_index()

X_train_cat = X_train.select_dtypes(np.object)
X_test_cat = X_test.select_dtypes (np.object)

In [34]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(X_train_cat)
cols = encoder.get_feature_names_out(input_features=X_train_cat.columns)
X_train_cat_encoded = pd.DataFrame(encoder.transform(X_train_cat).toarray(),columns=cols)
X_train_cat_encoded.head()

STATE_FL  STATE_GA  STATE_IL  STATE_IN  STATE_MI  STATE_MO  STATE_NC  \
0       1.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       1.0       0.0       0.0       0.0       0.0       0.0       0.0   
4       0.0       0.0       1.0       0.0       0.0       0.0       0.0   

   STATE_TX  STATE_WA  STATE_WI  STATE_other  HOMEOWNR_U  GENDER_M  \
0       0.0       0.0       0.0          0.0         0.0       1.0   
1       0.0       0.0       0.0          1.0         0.0       0.0   
2       0.0       0.0       0.0          0.0         0.0       0.0   
3       0.0       0.0       0.0          0.0         1.0       0.0   
4       0.0       0.0       0.0          0.0         0.0       1.0   

   GENDER_other  RFA_2A_E  RFA_2A_F  RFA_2A_G  DOMAIN_A_R  DOMAIN_A_S  \
0           0.0       0.0       1.0       0.0         0.0         0.0   
1           0.0       0.0       1.0       0.0         0.0         0.0   
2           0.0       0.0       1.0       0.0         0.0         0.0   
3           0.0       1.0       0.0       0.0         0.0         0.0   
4           0.0       0.0       1.0       0.0         0.0         0.0   

   DOMAIN_A_T  DOMAIN_A_U  
0         1.0         0.0  
1         0.0         1.0  
2         1.0         0.0  
3         0.0         0.0  
4         1.0         0.0

In [41]:
encoder_test_cat = encoder.transform(X_test_cat)
cols = encoder.get_feature_names_out(input_features=X_test_cat.columns)
X_test_cat_encoded = pd.DataFrame(encoder.transform(X_test_cat).toarray(),columns=cols)
X_test_cat_encoded.head()

STATE_FL  STATE_GA  STATE_IL  STATE_IN  STATE_MI  STATE_MO  STATE_NC  \
0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       1.0       0.0   
2       0.0       0.0       1.0       0.0       0.0       0.0       0.0   
3       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4       0.0       0.0       0.0       0.0       0.0       0.0       0.0   

   STATE_TX  STATE_WA  STATE_WI  STATE_other  HOMEOWNR_U  GENDER_M  \
0       0.0       0.0       0.0          1.0         0.0       1.0   
1       0.0       0.0       0.0          0.0         0.0       0.0   
2       0.0       0.0       0.0          0.0         1.0       1.0   
3       0.0       0.0       0.0          0.0         0.0       1.0   
4       1.0       0.0       0.0          0.0         1.0       0.0   

   GENDER_other  RFA_2A_E  RFA_2A_F  RFA_2A_G  DOMAIN_A_R  DOMAIN_A_S  \
0           0.0       0.0       0.0       1.0         0.0         0.0   
1           0.0       0.0       1.0       0.0         1.0         0.0   
2           0.0       0.0       0.0       1.0         1.0         0.0   
3           0.0       0.0       1.0       0.0         0.0         1.0   
4           0.0       0.0       1.0       0.0         0.0         0.0   

   DOMAIN_A_T  DOMAIN_A_U  
0         1.0         0.0  
1         0.0         0.0  
2         0.0         0.0  
3         0.0         0.0  
4         0.0         0.0

Concatenating both the scaled numerical data and train one hot encoded data

In [37]:
X_train_new = pd.concat([X_train_cat_encoded,X_train_num],axis =1)
X_train_new

STATE_FL  STATE_GA  STATE_IL  STATE_IN  STATE_MI  STATE_MO  STATE_NC  \
0            1.0       0.0       0.0       0.0       0.0       0.0       0.0   
1            0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2            0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3            1.0       0.0       0.0       0.0       0.0       0.0       0.0   
4            0.0       0.0       1.0       0.0       0.0       0.0       0.0   
...          ...       ...       ...       ...       ...       ...       ...   
115972       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
115973       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
115974       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
115975       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
115976       1.0       0.0       0.0       0.0       0.0       0.0       0.0   

        STATE_TX  STATE_WA  STATE_WI  STATE_other  HOMEOWNR_U  GENDER_M  \
0            0.0       0.0       0.0          0.0         0.0       1.0   
1            0.0       0.0       0.0          1.0         0.0       0.0   
2            0.0       0.0       0.0          0.0         0.0       0.0   
3            0.0       0.0       0.0          0.0         1.0       0.0   
4            0.0       0.0       0.0          0.0         0.0       1.0   
...          ...       ...       ...          ...         ...       ...   
115972       0.0       0.0       0.0          0.0         1.0       0.0   
115973       0.0       0.0       0.0          1.0         0.0       1.0   
115974       0.0       0.0       0.0          0.0         0.0       0.0   
115975       1.0       0.0       0.0          0.0         1.0       0.0   
115976       0.0       0.0       0.0          0.0         0.0       0.0   

        GENDER_other  RFA_2A_E  RFA_2A_F  RFA_2A_G  DOMAIN_A_R  DOMAIN_A_S  \
0                0.0       0.0       1.0       0.0         0.0         0.0   
1                0.0       0.0       1.0       0.0         0.0         0.0   
2                0.0       0.0       1.0       0.0         0.0         0.0   
3                0.0       1.0       0.0       0.0         0.0         0.0   
4                0.0       0.0       1.0       0.0         0.0         0.0   
...              ...       ...       ...       ...         ...         ...   
115972           0.0       0.0       0.0       0.0         0.0         0.0   
115973           0.0       1.0       0.0       0.0         0.0         1.0   
115974           0.0       0.0       1.0       0.0         0.0         0.0   
115975           0.0       0.0       1.0       0.0         0.0         0.0   
115976           0.0       0.0       1.0       0.0         0.0         0.0   

        DOMAIN_A_T  DOMAIN_A_U  index  TCODE        AGE  INCOME  WEALTH1  HIT  \
0              1.0         0.0  47291      1  61.611649       3        3    0   
1              0.0         1.0  23413      0  66.000000       5        9    0   
2              1.0         0.0  14312     28  49.000000       6        7    1   
3              0.0         0.0  74320      0  76.000000       2        7    8   
4              1.0         0.0  20185      1  64.000000       5        1    2   
...            ...         ...    ...    ...        ...     ...      ...  ...   
115972         0.0         0.0    384      0  87.000000       1        2    2   
115973         0.0         0.0  84551      1  77.000000       4        6   11   
115974         0.0         1.0  41947      0  68.000000       5        6    1   
115975         0.0         1.0   7766      0  33.000000       4        9    0   
115976         0.0         1.0  30768      0  50.000000       4        9    0   

        MALEMILI  MALEVET  VIETVETS  WWIIVETS  LOCALGOV  STATEGOV  FEDGOV  \
0              0       52        23        48        19         1       0   
1              2       25        29        31         8         1       2   
2    

In [42]:
X_test_new = pd.concat([X_test_cat_encoded,X_test_num],axis =1)
X_test_new

STATE_FL  STATE_GA  STATE_IL  STATE_IN  STATE_MI  STATE_MO  STATE_NC  \
0           0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1           0.0       0.0       0.0       0.0       0.0       1.0       0.0   
2           0.0       0.0       1.0       0.0       0.0       0.0       0.0   
3           0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4           0.0       0.0       0.0       0.0       0.0       0.0       0.0   
...         ...       ...       ...       ...       ...       ...       ...   
28990       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
28991       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
28992       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
28993       0.0       0.0       1.0       0.0       0.0       0.0       0.0   
28994       0.0       0.0       1.0       0.0       0.0       0.0       0.0   

       STATE_TX  STATE_WA  STATE_WI  STATE_other  HOMEOWNR_U  GENDER_M  \
0           0.0       0.0       0.0          1.0         0.0       1.0   
1           0.0       0.0       0.0          0.0         0.0       0.0   
2           0.0       0.0       0.0          0.0         1.0       1.0   
3           0.0       0.0       0.0          0.0         0.0       1.0   
4           1.0       0.0       0.0          0.0         1.0       0.0   
...         ...       ...       ...          ...         ...       ...   
28990       0.0       0.0       0.0          1.0         0.0       0.0   
28991       0.0       0.0       0.0          1.0         0.0       0.0   
28992       0.0       1.0       0.0          0.0         0.0       1.0   
28993       0.0       0.0       0.0          0.0         0.0       1.0   
28994       0.0       0.0       0.0          0.0         0.0       0.0   

       GENDER_other  RFA_2A_E  RFA_2A_F  RFA_2A_G  DOMAIN_A_R  DOMAIN_A_S  \
0               0.0       0.0       0.0       1.0         0.0         0.0   
1               0.0       0.0       1.0       0.0         1.0         0.0   
2               0.0       0.0       0.0       1.0         1.0         0.0   
3               0.0       0.0       1.0       0.0         0.0         1.0   
4               0.0       0.0       1.0       0.0         0.0         0.0   
...             ...       ...       ...       ...         ...         ...   
28990           1.0       0.0       1.0       0.0         0.0         0.0   
28991           0.0       0.0       0.0       0.0         1.0         0.0   
28992           0.0       0.0       1.0       0.0         0.0         0.0   
28993           0.0       0.0       1.0       0.0         1.0         0.0   
28994           0.0       0.0       1.0       0.0         0.0         0.0   

       DOMAIN_A_T  DOMAIN_A_U  index  TCODE        AGE  INCOME  WEALTH1  HIT  \
0             1.0         0.0  81286      2  84.000000       5        7   31   
1             0.0         0.0  74499      2  61.611649       5        9    0   
2             0.0         0.0  86937      1  37.000000       1        1    1   
3             0.0         0.0  45164   4002  77.000000       5        2    9   
4             0.0         0.0  65166      0  47.000000       6        9    3   
...           ...         ...    ...    ...        ...     ...      ...  ...   
28990         1.0         0.0  71581      0  61.611649       4        9    0   
28991         0.0         0.0  38253      2  67.000000       2        0    0   
28992         1.0         0.0  15975      1  51.000000       4        1   25   
28993         0.0         0.0  74333      1  67.000000       5        8    2   
28994         1.0         0.0  31217      0  42.000000       4        5    8   

       MALEMILI  MALEVET  VIETVETS  WWIIVETS  LOCALGOV  STATEGOV  FEDGOV  \
0             0       31        28        54         3        25       4   
1             0       29        41        33         4         3       4   
2             0       34        30        43         5 

##### RandomForestClassifier

Preparing the Random Forest Classifier on deleted categorical columns but unchanged numerical columns 

In [45]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5, #max number of questions to ask
                             min_samples_split=20, #amount of rows still considered at every question
                             min_samples_leaf =20, #ultimate answer based on at least this many rows
                             max_samples=0.2, # fraction of X_train to use in each tree
                             random_state = 42)
clf.fit(X_train_new, y_train)
print(clf.score(X_train_new, y_train))
print(clf.score(X_test_new, y_test))

y_pred = clf.predict(X_test_new)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.6172258292592496
0.6148991205380238


1    14693
0    14302
Name: TARGET_B, dtype: int64

array([[8820, 5482],
       [5684, 9009]])

In [46]:
# For cross validation
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20)
cross_val_scores = cross_val_score(clf, X_train_new, y_train, cv=10)
print(cross_val_scores.mean())

0.6179329020564943


MinMax scaling for the Variance Threshold feature selection. Yesterday's lab I did the KBest, therefore trying out a different method

In [47]:
X_num_train = X_train_new.select_dtypes(np.number).reset_index()
X_num_test = X_test_new.select_dtypes(np.number).reset_index()

X_cat_train = X_train_new.select_dtypes(np.object)
X_cat_test = X_test_new.select_dtypes (np.object)

In [48]:
from sklearn.preprocessing import MinMaxScaler
transformer = MinMaxScaler().fit(X_num_train)
x_normalized = transformer.transform(X_num_train)
x_norm = pd.DataFrame(x_normalized, columns=X_num_train.columns)
x_norm.head()
#this is should fit X_train
# just transform X_test

level_0  STATE_FL  STATE_GA  STATE_IL  STATE_IN  STATE_MI  STATE_MO  \
0  0.000000       1.0       0.0       0.0       0.0       0.0       0.0   
1  0.000009       0.0       0.0       0.0       0.0       0.0       0.0   
2  0.000017       0.0       0.0       0.0       0.0       0.0       0.0   
3  0.000026       1.0       0.0       0.0       0.0       0.0       0.0   
4  0.000034       0.0       0.0       1.0       0.0       0.0       0.0   

   STATE_NC  STATE_TX  STATE_WA  STATE_WI  STATE_other  HOMEOWNR_U  GENDER_M  \
0       0.0       0.0       0.0       0.0          0.0         0.0       1.0   
1       0.0       0.0       0.0       0.0          1.0         0.0       0.0   
2       0.0       0.0       0.0       0.0          0.0         0.0       0.0   
3       0.0       0.0       0.0       0.0          0.0         1.0       0.0   
4       0.0       0.0       0.0       0.0          0.0         0.0       1.0   

   GENDER_other  RFA_2A_E  RFA_2A_F  RFA_2A_G  DOMAIN_A_R  DOMAIN_A_S  \
0           0.0       0.0       1.0       0.0         0.0         0.0   
1           0.0       0.0       1.0       0.0         0.0         0.0   
2           0.0       0.0       1.0       0.0         0.0         0.0   
3           0.0       1.0       0.0       0.0         0.0         0.0   
4           0.0       0.0       1.0       0.0         0.0         0.0   

   DOMAIN_A_T  DOMAIN_A_U     index     TCODE       AGE    INCOME   WEALTH1  \
0         1.0         0.0  0.495650  0.000017  0.624862  0.333333  0.333333   
1         0.0         1.0  0.245383  0.000000  0.670103  0.666667  1.000000   
2         1.0         0.0  0.149995  0.000483  0.494845  0.833333  0.777778   
3         0.0         0.0  0.778944  0.000000  0.773196  0.166667  0.777778   
4         1.0         0.0  0.211550  0.000017  0.649485  0.666667  0.111111   

        HIT  MALEMILI   MALEVET  VIETVETS  WWIIVETS  LOCALGOV  STATEGOV  \
0  0.000000  0.000000  0.525253  0.232323  0.484848  0.191919  0.010101   
1  0.000000  0.020202  0.252525  0.292929  0.313131  0.080808  0.010101   
2  0.004149  0.000000  0.414141  0.232323  0.565657  0.090909  0.181818   
3  0.033195  0.060606  0.333333  0.313131  0.333333  0.040404  0.000000   
4  0.008299  0.000000  0.313131  0.282828  0.515152  0.040404  0.070707   

     FEDGOV   WEALTH2    POP901    POP902    POP903   POP90C1   POP90C2  \
0  0.000000  1.000000  0.008642  0.013176  0.012586  0.000000  0.000000   
1  0.022989  1.000000  0.070546  0.072042  0.068491  0.000000  0.989899   
2  0.000000  1.000000  0.008652  0.010163  0.009922  0.000000  0.000000   
3  0.091954  0.777778  0.013151  0.018618  0.014898  1.000000  0.000000   
4  0.011494  0.111111  0.015623  0.020506  0.017269  0.707071  0.000000   

    POP90C3   POP90C4   POP90C5      ETH1      ETH2      ETH3      ETH4  \
0  1.000000  0.484848  0.525253  1.000000  0.000000  0.000000  0.000000   
1  0.020202  0.555556  0.454545  0.878788  0.050505  0.020202  0.020202   
2  1.000000  0.565657  0.444444  0.898990  0.010101  0.010101  0.010101   
3  0.000000  0.494949  0.515152  0.969697  0.020202  0.000000  0.010101   
4  0.303030  0.484848  0.525253  1.000000  0.000000  0.000000  0.000000   

       ETH5  ETH6  ETH7  ETH8      ETH9  ETH10  ETH11  ETH12     ETH13  \
0  0.010101   0.0   0.0   0.0  0.000000    0.0    0.0    0.0  0.010309   
1  0.101010   0.0   0.0   0.0  0.000000    0.0    0.0    0.0  0.072165   
2  0.191919   0.0   0.0   0.0  0.014925    0.0    0.0    0.0  0.185567   
3  0.080808   0.0   0.0   0.0  0.000000    0.0    0.0    0.0  0.010309   
4  0.010101   0.0   0.0   0.0  0.000000    0.0    0.0    0.0  0.010309   

      ETH14  ETH15     ETH16    AGE901    AGE902    AGE903    AGE904  \
0  0.000000    0.0  0.000000  0.722892  0.746988  0.746988  0.654321   
1  0.000000    0.0  0.034884  0.385542  0.445783  0.481928  0.419753   
2  0.000000    0.0  0.011628  0.481928  0.530120  0.542169  0.493827   
3  0.081633    0.0  0.023256  0.421687  0.518072  0.554217  0.469136   


In [49]:
X_test_normalized = transformer.transform(X_num_test)
X_test_norm = pd.DataFrame(X_test_normalized, columns = X_num_train.columns)

#### Variance Threshold

In [50]:
from sklearn.feature_selection import VarianceThreshold 
var_threshold = 0.02
sel = VarianceThreshold(threshold=(var_threshold))

# 1) default is threshold = 0 :eliminate all coumns that are identical for all rows
# 2) in practice we would scale the columns first, and then apply threshold, or apply 
#    a different threshold for different columns

# This drops the columns that have a variance less than this threshold
sel = sel.fit(x_norm)
temp = sel.transform(x_norm)
temp = pd.DataFrame(temp)
print(x_norm.shape)
print(temp.shape)
#use the variace thre on X_train scale

(115977, 341)
(115977, 104)


In [51]:
sel.variances_ > var_threshold
sel.get_support()
var_list = list(sel.get_support())

In [52]:
[col[0] for col in zip(numerical.columns, var_list) if col[1] == False]

['ETH3',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH16',
 'AGE901',
 'AGE902',
 'AGE906',
 'AGE907',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN3',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW1',
 'DW2',
 'DW3',
 'DW4',
 'DW5',
 'DW6',
 'DW7',
 'DW8',
 'DW9',
 'HV1',
 'HV2',
 'HV3',
 'HV4',
 'HU2',
 'HU3',
 'HU4',
 'HU5',
 'HHD1',
 'HHD2',
 'HHD3',
 'HHD4',
 'HHD5',
 'HHD8',
 'HHD12',
 'ETHC1',
 'ETHC2',
 'HVP3',
 'HVP4',
 'HVP6',
 'RHP1',
 'RHP2',
 'RHP3',
 'RHP4',
 'HUPA1',
 'HUPA2',
 'HUPA3',
 'HUPA4',
 'HUPA5',
 'HUPA6',
 'HUPA7',
 'RP1',
 'RP2',
 'RP3',
 'RP4',
 'IC4',
 'IC6',
 'IC7',
 'IC8',
 'IC9',
 'IC10',
 'IC13',
 'IC14',
 'IC16',
 'IC23',
 'HHAS1',
 'HHAS2',
 'HHAS3',
 'HHAS4',
 'MC1',
 'MC3',
 'TPE1',
 'TPE2',
 'TPE3',
 'TPE4',
 'TPE5',
 'TPE6',
 'TPE7',
 'TPE8',
 '

Deleting these recommended columns

In [53]:
drop_col = ['AGE',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'STATEGOV',
 'FEDGOV',
 'WEALTH2',
 'POP902',
 'POP903',
 'POP90C1',
 'POP90C5',
 'ETH3',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH16',
 'AGE901',
 'AGE902',
 'AGE906',
 'AGE907',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN3',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW1',
 'DW2',
 'DW3',
 'DW4',
 'DW5',
 'DW6',
 'DW7',
 'DW8',
 'DW9',
 'HV1',
 'HV2',
 'HV3',
 'HV4',
 'HU2',
 'HU3',
 'HU4',
 'HU5',
 'HHD1',
 'HHD2',
 'HHD3',
 'HHD4',
 'HHD5',
 'HHD8',
 'HHD12',
 'ETHC1',
 'ETHC2',
 'HVP3',
 'HVP4',
 'HVP6',
 'RHP1',
 'RHP2',
 'RHP3',
 'RHP4',
 'HUPA1',
 'HUPA2',
 'HUPA3',
 'HUPA4',
 'HUPA5',
 'HUPA6',
 'HUPA7',
 'RP1',
 'RP2',
 'RP3',
 'RP4',
 'IC4',
 'IC6',
 'IC7',
 'IC8',
 'IC9',
 'IC10',
 'IC13',
 'IC14',
 'IC16',
 'IC23',
 'HHAS1',
 'HHAS2',
 'HHAS3',
 'HHAS4',
 'MC1',
 'MC3',
 'TPE1',
 'TPE2',
 'TPE3',
 'TPE4',
 'TPE5',
 'TPE6',
 'TPE7',
 'TPE8',
 'TPE9',
 'PEC1',
 'PEC2',
 'TPE10',
 'TPE11',
 'TPE12',
 'TPE13',
 'LFC1',
 'LFC3',
 'LFC5',
 'LFC8',
 'LFC9',
 'LFC10',
 'OCC1',
 'OCC2',
 'OCC3',
 'OCC4',
 'OCC5',
 'OCC6',
 'OCC7',
 'OCC8',
 'OCC10',
 'OCC11',
 'OCC12',
 'EIC1',
 'EIC3',
 'EIC5',
 'EIC10',
 'EIC11',
 'EIC12',
 'EIC13',
 'EIC14',
 'EIC15',
 'EIC16',
 'OEDC1',
 'OEDC2',
 'OEDC3',
 'OEDC4',
 'OEDC5',
 'OEDC6',
 'OEDC7',
 'EC1',
 'EC2',
 'EC3',
 'EC4',
 'EC5',
 'EC6',
 'EC7',
 'EC8',
 'SEC1',
 'SEC2',
 'SEC3',
 'SEC4',
 'SEC5',
 'AFC1',
 'AFC2',
 'AFC3',
 'AFC4',
 'AFC5',
 'AFC6',
 'VC1',
 'VC2',
 'VC3',
 'VC4',
 'ANC1',
 'ANC2',
 'ANC3',
 'ANC4',
 'ANC5',
 'ANC6',
 'ANC7',
 'ANC8',
 'ANC9',
 'ANC10',
 'ANC11',
 'ANC12',
 'ANC13',
 'ANC14',
 'ANC15',
 'POBC1',
 'POBC2',
 'LSC1',
 'LSC2',
 'LSC4',
 'VOC2',
 'VOC3',
 'HC1',
 'HC2',
 'HC3',
 'HC4',
 'HC5',
 'HC6',
 'HC7',
 'HC8',
 'HC9',
 'HC10',
 'HC11',
 'HC12',
 'HC13',
 'HC14',
 'HC15',
 'HC18',
 'HC19',
 'HC20',
 'HC21',
 'MHUC2',
 'AC1',
 'CARDPROM',
 'CARDGIFT',
 'MINRAMNT',
 'LASTGIFT',
 'AVGGIFT',
 'CONTROLN',
 'HPHONE_D']

Dropping these values from the train and test set

In [54]:
X_num = x_norm.drop(columns=drop_col, axis=1)
X_num

level_0  STATE_FL  STATE_GA  STATE_IL  STATE_IN  STATE_MI  STATE_MO  \
0       0.000000       1.0       0.0       0.0       0.0       0.0       0.0   
1       0.000009       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.000017       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.000026       1.0       0.0       0.0       0.0       0.0       0.0   
4       0.000034       0.0       0.0       1.0       0.0       0.0       0.0   
...          ...       ...       ...       ...       ...       ...       ...   
115972  0.999966       0.0       0.0       0.0       0.0       0.0       0.0   
115973  0.999974       0.0       0.0       0.0       0.0       0.0       0.0   
115974  0.999983       0.0       0.0       0.0       0.0       0.0       0.0   
115975  0.999991       0.0       0.0       0.0       0.0       0.0       0.0   
115976  1.000000       1.0       0.0       0.0       0.0       0.0       0.0   

        STATE_NC  STATE_TX  STATE_WA  STATE_WI  STATE_other  HOMEOWNR_U  \
0            0.0       0.0       0.0       0.0          0.0         0.0   
1            0.0       0.0       0.0       0.0          1.0         0.0   
2            0.0       0.0       0.0       0.0          0.0         0.0   
3            0.0       0.0       0.0       0.0          0.0         1.0   
4            0.0       0.0       0.0       0.0          0.0         0.0   
...          ...       ...       ...       ...          ...         ...   
115972       0.0       0.0       0.0       0.0          0.0         1.0   
115973       0.0       0.0       0.0       0.0          1.0         0.0   
115974       0.0       0.0       0.0       0.0          0.0         0.0   
115975       0.0       1.0       0.0       0.0          0.0         1.0   
115976       0.0       0.0       0.0       0.0          0.0         0.0   

        GENDER_M  GENDER_other  RFA_2A_E  RFA_2A_F  RFA_2A_G  DOMAIN_A_R  \
0            1.0           0.0       0.0       1.0       0.0         0.0   
1            0.0           0.0       0.0       1.0       0.0         0.0   
2            0.0           0.0       0.0       1.0       0.0         0.0   
3            0.0           0.0       1.0       0.0       0.0         0.0   
4            1.0           0.0       0.0       1.0       0.0         0.0   
...          ...           ...       ...       ...       ...         ...   
115972       0.0           0.0       0.0       0.0       0.0         0.0   
115973       1.0           0.0       1.0       0.0       0.0         0.0   
115974       0.0           0.0       0.0       1.0       0.0         0.0   
115975       0.0           0.0       0.0       1.0       0.0         0.0   
115976       0.0           0.0       0.0       1.0       0.0         0.0   

        DOMAIN_A_S  DOMAIN_A_T  DOMAIN_A_U     index     TCODE    INCOME  \
0              0.0         1.0         0.0  0.495650  0.000017  0.333333   
1              0.0         0.0         1.0  0.245383  0.000000  0.666667   
2              0.0         1.0         0.0  0.149995  0.000483  0.833333   
3              0.0         0.0         0.0  0.778944  0.000000  0.166667   
4              0.0         1.0         0.0  0.211550  0.000017  0.666667   
...            ...         ...         ...       ...       ...       ...   
115972         0.0         0.0         0.0  0.004014  0.000000  0.000000   
115973         1.0         0.0         0.0  0.886175  0.000017  0.500000   
115974         0.0         0.0         1.0  0.439639  0.000000  0.666667   
115975         0.0         0.0         1.0  0.081386  0.000000  0.500000   
115976         0.0         0.0         1.0  0.322471  0.000000  0.500000   

         WEALTH1       HIT  WWIIVETS  LOCALGOV    POP901   POP90C2   POP90C3  \
0       0.333333  0.000000  0.484848  0.191919  0.008642  0.000000  1.000000   
1       1.000000  0.000000  0.313131  0.080808  0.070546  0.989899  0.020202   
2       0.777778  0.004149  0.565657  0.090909  0.008652  0.000000  1.000000   
3

In [55]:
X_norm_test = X_test_norm.drop(columns=drop_col, axis=1)
X_norm_test

level_0  STATE_FL  STATE_GA  STATE_IL  STATE_IN  STATE_MI  STATE_MO  \
0      0.000000       0.0       0.0       0.0       0.0       0.0       0.0   
1      0.000009       0.0       0.0       0.0       0.0       0.0       1.0   
2      0.000017       0.0       0.0       1.0       0.0       0.0       0.0   
3      0.000026       0.0       0.0       0.0       0.0       0.0       0.0   
4      0.000034       0.0       0.0       0.0       0.0       0.0       0.0   
...         ...       ...       ...       ...       ...       ...       ...   
28990  0.249966       0.0       0.0       0.0       0.0       0.0       0.0   
28991  0.249974       0.0       0.0       0.0       0.0       0.0       0.0   
28992  0.249983       0.0       0.0       0.0       0.0       0.0       0.0   
28993  0.249991       0.0       0.0       1.0       0.0       0.0       0.0   
28994  0.250000       0.0       0.0       1.0       0.0       0.0       0.0   

       STATE_NC  STATE_TX  STATE_WA  STATE_WI  STATE_other  HOMEOWNR_U  \
0           0.0       0.0       0.0       0.0          1.0         0.0   
1           0.0       0.0       0.0       0.0          0.0         0.0   
2           0.0       0.0       0.0       0.0          0.0         1.0   
3           0.0       0.0       0.0       0.0          0.0         0.0   
4           0.0       1.0       0.0       0.0          0.0         1.0   
...         ...       ...       ...       ...          ...         ...   
28990       0.0       0.0       0.0       0.0          1.0         0.0   
28991       0.0       0.0       0.0       0.0          1.0         0.0   
28992       0.0       0.0       1.0       0.0          0.0         0.0   
28993       0.0       0.0       0.0       0.0          0.0         0.0   
28994       0.0       0.0       0.0       0.0          0.0         0.0   

       GENDER_M  GENDER_other  RFA_2A_E  RFA_2A_F  RFA_2A_G  DOMAIN_A_R  \
0           1.0           0.0       0.0       0.0       1.0         0.0   
1           0.0           0.0       0.0       1.0       0.0         1.0   
2           1.0           0.0       0.0       0.0       1.0         1.0   
3           1.0           0.0       0.0       1.0       0.0         0.0   
4           0.0           0.0       0.0       1.0       0.0         0.0   
...         ...           ...       ...       ...       ...         ...   
28990       0.0           1.0       0.0       1.0       0.0         0.0   
28991       0.0           0.0       0.0       0.0       0.0         1.0   
28992       1.0           0.0       0.0       1.0       0.0         0.0   
28993       1.0           0.0       0.0       1.0       0.0         1.0   
28994       0.0           0.0       0.0       1.0       0.0         0.0   

       DOMAIN_A_S  DOMAIN_A_T  DOMAIN_A_U     index     TCODE    INCOME  \
0             0.0         1.0         0.0  0.851955  0.000034  0.666667   
1             0.0         0.0         0.0  0.780820  0.000034  0.666667   
2             0.0         0.0         0.0  0.911183  0.000017  0.000000   
3             1.0         0.0         0.0  0.473357  0.068998  0.666667   
4             0.0         0.0         0.0  0.683000  0.000000  0.833333   
...           ...         ...         ...       ...       ...       ...   
28990         0.0         1.0         0.0  0.750236  0.000000  0.500000   
28991         0.0         0.0         0.0  0.400922  0.000034  0.166667   
28992         0.0         1.0         0.0  0.167425  0.000017  0.500000   
28993         0.0         0.0         0.0  0.779080  0.000017  0.666667   
28994         0.0         1.0         0.0  0.327177  0.000000  0.500000   

        WEALTH1       HIT  WWIIVETS  LOCALGOV    POP901  POP90C2   POP90C3  \
0      0.777778  0.128631  0.545455  0.030303  0.005593      1.0  0.000000   
1      1.000000  0.000000  0.333333  0.040404  0.063191      0.0  1.000000   
2      0.111111  0.004149  0.434343  0.050505  0.012492      0.0  1.000000   
3      0.222222  0.037344  0.666667  0.000000  0.004276 

Scaling the data again for the Logistic Regression and Decision Tree Classifiers

In [70]:
Xtrain_transformed = pd.concat([X_num, X_cat_train], axis = 1)
Xtrain_transformed.head(10)
#concat X train num and cat
# concat X test num and cat

level_0  STATE_FL  STATE_GA  STATE_IL  STATE_IN  STATE_MI  STATE_MO  \
0  0.000000       1.0       0.0       0.0       0.0       0.0       0.0   
1  0.000009       0.0       0.0       0.0       0.0       0.0       0.0   
2  0.000017       0.0       0.0       0.0       0.0       0.0       0.0   
3  0.000026       1.0       0.0       0.0       0.0       0.0       0.0   
4  0.000034       0.0       0.0       1.0       0.0       0.0       0.0   
5  0.000043       0.0       0.0       0.0       0.0       0.0       0.0   
6  0.000052       0.0       0.0       0.0       0.0       0.0       0.0   
7  0.000060       0.0       0.0       0.0       0.0       1.0       0.0   
8  0.000069       0.0       0.0       0.0       0.0       0.0       0.0   
9  0.000078       0.0       0.0       1.0       0.0       0.0       0.0   

   STATE_NC  STATE_TX  STATE_WA  STATE_WI  STATE_other  HOMEOWNR_U  GENDER_M  \
0       0.0       0.0       0.0       0.0          0.0         0.0       1.0   
1       0.0       0.0       0.0       0.0          1.0         0.0       0.0   
2       0.0       0.0       0.0       0.0          0.0         0.0       0.0   
3       0.0       0.0       0.0       0.0          0.0         1.0       0.0   
4       0.0       0.0       0.0       0.0          0.0         0.0       1.0   
5       1.0       0.0       0.0       0.0          0.0         1.0       1.0   
6       0.0       0.0       0.0       0.0          1.0         0.0       0.0   
7       0.0       0.0       0.0       0.0          0.0         0.0       1.0   
8       0.0       0.0       0.0       1.0          0.0         0.0       0.0   
9       0.0       0.0       0.0       0.0          0.0         0.0       0.0   

   GENDER_other  RFA_2A_E  RFA_2A_F  RFA_2A_G  DOMAIN_A_R  DOMAIN_A_S  \
0           0.0       0.0       1.0       0.0         0.0         0.0   
1           0.0       0.0       1.0       0.0         0.0         0.0   
2           0.0       0.0       1.0       0.0         0.0         0.0   
3           0.0       1.0       0.0       0.0         0.0         0.0   
4           0.0       0.0       1.0       0.0         0.0         0.0   
5           0.0       0.0       1.0       0.0         0.0         0.0   
6           0.0       0.0       0.0       1.0         0.0         0.0   
7           0.0       0.0       0.0       0.0         1.0         0.0   
8           0.0       1.0       0.0       0.0         1.0         0.0   
9           0.0       0.0       1.0       0.0         0.0         0.0   

   DOMAIN_A_T  DOMAIN_A_U     index     TCODE    INCOME   WEALTH1       HIT  \
0         1.0         0.0  0.495650  0.000017  0.333333  0.333333  0.000000   
1         0.0         1.0  0.245383  0.000000  0.666667  1.000000  0.000000   
2         1.0         0.0  0.149995  0.000483  0.833333  0.777778  0.004149   
3         0.0         0.0  0.778944  0.000000  0.166667  0.777778  0.033195   
4         1.0         0.0  0.211550  0.000017  0.666667  0.111111  0.008299   
5         1.0         0.0  0.176082  0.000017  0.333333  1.000000  0.000000   
6         1.0         0.0  0.414904  0.000000  0.666667  1.000000  0.000000   
7         0.0         0.0  0.945540  0.000017  0.500000  1.000000  0.000000   
8         0.0         0.0  0.323090  0.000000  0.833333  1.000000  0.000000   
9         0.0         0.0  0.913552  0.000034  1.000000  1.000000  0.000000   

   WWIIVETS  LOCALGOV    POP901   POP90C2   POP90C3   POP90C4      ETH1  \
0  0.484848  0.191919  0.008642  0.000000  1.000000  0.484848  1.000000   
1  0.313131  0.080808  0.070546  0.989899  0.020202  0.555556  0.878788   
2  0.565657  0.090909  0.008652  0.000000  1.000000  0.565657  0.898990   
3  0.333333  0.040404  0.013151  0.000000  0.000000  0.494949  0.969697   
4  0.515152  0.040404  0.015623  0.000000  0.303030  0.484848  1.000000   
5  0.343434  0.050505  0.002472  0.262626  0.747475  0.484848  0.858586   
6  0.383838  0.090909  0.010527  0.000000  0.202020  0.505051  0.898990   
7  0.404040  0.101010  

In [71]:
Xtest_transformed = pd.concat([X_norm_test, X_cat_test], axis = 1)
Xtest_transformed.head(10)

level_0  STATE_FL  STATE_GA  STATE_IL  STATE_IN  STATE_MI  STATE_MO  \
0  0.000000       0.0       0.0       0.0       0.0       0.0       0.0   
1  0.000009       0.0       0.0       0.0       0.0       0.0       1.0   
2  0.000017       0.0       0.0       1.0       0.0       0.0       0.0   
3  0.000026       0.0       0.0       0.0       0.0       0.0       0.0   
4  0.000034       0.0       0.0       0.0       0.0       0.0       0.0   
5  0.000043       0.0       0.0       0.0       0.0       0.0       0.0   
6  0.000052       1.0       0.0       0.0       0.0       0.0       0.0   
7  0.000060       0.0       0.0       0.0       0.0       0.0       0.0   
8  0.000069       0.0       0.0       0.0       0.0       0.0       0.0   
9  0.000078       0.0       0.0       1.0       0.0       0.0       0.0   

   STATE_NC  STATE_TX  STATE_WA  STATE_WI  STATE_other  HOMEOWNR_U  GENDER_M  \
0       0.0       0.0       0.0       0.0          1.0         0.0       1.0   
1       0.0       0.0       0.0       0.0          0.0         0.0       0.0   
2       0.0       0.0       0.0       0.0          0.0         1.0       1.0   
3       0.0       0.0       0.0       0.0          0.0         0.0       1.0   
4       0.0       1.0       0.0       0.0          0.0         1.0       0.0   
5       0.0       1.0       0.0       0.0          0.0         0.0       0.0   
6       0.0       0.0       0.0       0.0          0.0         0.0       0.0   
7       0.0       0.0       0.0       0.0          0.0         0.0       1.0   
8       0.0       0.0       0.0       0.0          0.0         0.0       0.0   
9       0.0       0.0       0.0       0.0          0.0         0.0       1.0   

   GENDER_other  RFA_2A_E  RFA_2A_F  RFA_2A_G  DOMAIN_A_R  DOMAIN_A_S  \
0           0.0       0.0       0.0       1.0         0.0         0.0   
1           0.0       0.0       1.0       0.0         1.0         0.0   
2           0.0       0.0       0.0       1.0         1.0         0.0   
3           0.0       0.0       1.0       0.0         0.0         1.0   
4           0.0       0.0       1.0       0.0         0.0         0.0   
5           0.0       0.0       0.0       1.0         0.0         0.0   
6           0.0       0.0       0.0       0.0         0.0         0.0   
7           0.0       0.0       1.0       0.0         0.0         0.0   
8           0.0       0.0       0.0       0.0         0.0         1.0   
9           0.0       0.0       0.0       0.0         0.0         1.0   

   DOMAIN_A_T  DOMAIN_A_U     index     TCODE    INCOME   WEALTH1       HIT  \
0         1.0         0.0  0.851955  0.000034  0.666667  0.777778  0.128631   
1         0.0         0.0  0.780820  0.000034  0.666667  1.000000  0.000000   
2         0.0         0.0  0.911183  0.000017  0.000000  0.111111  0.004149   
3         0.0         0.0  0.473357  0.068998  0.666667  0.222222  0.037344   
4         0.0         0.0  0.683000  0.000000  0.833333  1.000000  0.012448   
5         1.0         0.0  0.414516  0.000000  0.333333  1.000000  0.000000   
6         1.0         0.0  0.382297  0.000034  0.166667  0.111111  0.004149   
7         1.0         0.0  0.710355  0.000017  0.666667  1.000000  0.000000   
8         0.0         0.0  0.295095  0.000483  1.000000  0.888889  0.000000   
9         0.0         0.0  0.975904  0.000017  0.500000  0.555556  0.000000   

   WWIIVETS  LOCALGOV    POP901   POP90C2   POP90C3   POP90C4      ETH1  \
0  0.545455  0.030303  0.005593  1.000000  0.000000  0.474747  0.939394   
1  0.333333  0.040404  0.063191  0.000000  1.000000  0.505051  1.000000   
2  0.434343  0.050505  0.012492  0.000000  1.000000  0.515152  1.000000   
3  0.666667  0.000000  0.004276  0.000000  0.000000  0.515152  0.909091   
4  0.202020  0.050505  0.045724  0.000000  0.393939  0.525253  0.949495   
5  0.080808  0.111111  0.009362  0.101010  0.909091  0.515152  0.969697   
6  0.515152  0.080808  0.069847  0.000000  1.000000  0.494949  0.969697   
7  0.434343  0.222222  

#### Decision Tree Classifier and Logistic Regression Models

In [62]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [100]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=2)
model.fit(Xtrain_transformed, y_train)
model.score(Xtest_transformed,y_test)
predictions = model.predict(Xtest_transformed)

print(model.score(Xtest_transformed,y_test))

print(confusion_matrix(y_test,predictions))
print("precision: ",precision_score(y_test,predictions))
print("recall: ",recall_score(y_test,predictions))
print("f1: ",f1_score(y_test,predictions))

0.5650629418865322
[[10231  4071]
 [ 8540  6153]]
precision:  0.6018192488262911
recall:  0.4187708432586946
f1:  0.49387968053939074


In [103]:
from sklearn.metrics import accuracy_score

### Using different criterion to determine a better accuracy score, confusion matrix, precision, recall, and f1 scores

Entropy: 
Entropy is the measure of uncertainty of a random variable, it characterizes the impurity of an arbitrary collection of examples. The higher the entropy the more the information content.

In [106]:
clf_entropy = DecisionTreeClassifier(criterion = 'entropy',
                                    random_state = 100, max_depth=3,
                                    min_samples_leaf=5)
clf_entropy.fit(Xtrain_transformed,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=5,
                       random_state=100)

In [107]:
y_pred_en=clf_entropy.predict(Xtest_transformed)
y_pred_en

array([0, 0, 1, ..., 0, 1, 0])

In [108]:
print(('Accuracy is:'),accuracy_score(y_test,y_pred_en))

Accuracy is: 0.5723055699258492


In [116]:
print(confusion_matrix(y_test,y_pred_en))
print("precision: ",precision_score(y_test,y_pred_en))    # true pos / true pos+false pos
print("recall: ",recall_score(y_test,y_pred_en))    #recall is most important, true pos/true pos + false neg
print("f1: ",f1_score(y_test,y_pred_en))

[[ 6187  8115]
 [ 4286 10407]]
precision:  0.561872367994817
recall:  0.7082964677057102
f1:  0.626644588288424


Gini: Gini Index is a metric to measure how often a randomly chosen element would be incorrectly identified.
It means an attribute with lower gini index should be preferred.
Sklearn supports “gini” criteria for Gini Index and by default, it takes “gini” value.

In [109]:
clf_gini = DecisionTreeClassifier(criterion = 'gini',
                                    random_state = 100, max_depth=3,
                                    min_samples_leaf=5)
clf_gini.fit(Xtrain_transformed,y_train)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=5, random_state=100)

In [110]:
y_pred_en1=clf_gini.predict(Xtest_transformed)
y_pred_en1

array([0, 0, 1, ..., 0, 1, 0])

In [111]:
print(('Accuracy is:'),accuracy_score(y_test,y_pred_en1))

Accuracy is: 0.5723055699258492


In [115]:
print(confusion_matrix(y_test,y_pred_en1))
print("precision: ",precision_score(y_test,y_pred_en1))    # true pos / true pos+false pos
print("recall: ",recall_score(y_test,y_pred_en1))    #recall is most important, true pos/true pos + false neg
print("f1: ",f1_score(y_test,y_pred_en1))

[[ 6187  8115]
 [ 4286 10407]]
precision:  0.561872367994817
recall:  0.7082964677057102
f1:  0.626644588288424


_Conclusion_: Although the accuracy score are higher for the Gini and Entropy decision trees, I would still recommend using the original decision tree classifier and the true positives far outweigh the false positive and true negatives than the aforementioned 2. 

In [73]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(random_state=0, solver='saga')
LR.fit(Xtrain_transformed, y_train)
LR.score(Xtest_transformed, y_test) 

0.5859286083807553

In [74]:
pred = LR.predict(Xtest_transformed)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[8818, 5484],
       [6522, 8171]])

In [75]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


print("precision: ",precision_score(y_test,pred))    # true pos / true pos+false pos
print("recall: ",recall_score(y_test,pred))    #recall is most important, true pos/true pos + false neg
print("f1: ",f1_score(y_test,pred))

precision:  0.59838886854632
recall:  0.5561151568774246
f1:  0.5764780584168195


Had to drop the same columns that were in the numerical column as they weren't dropped in the test data

In [76]:
clf1 = RandomForestClassifier(max_depth=3, #max number of questions to ask
                             min_samples_split=20, #amount of rows still considered at every question
                             min_samples_leaf =20, #ultimate answer based on at least this many rows
                             max_samples=0.2, # fraction of X_train to use in each tree
                             random_state = 42)
clf1.fit(Xtrain_transformed, y_train)
print(clf1.score(Xtrain_transformed, y_train))
print(clf1.score(Xtest_transformed, y_test))

y_pred = clf1.predict(Xtest_transformed)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.5952990679186391
0.5921365752715986


1    14693
0    14302
Name: TARGET_B, dtype: int64

array([[8815, 5487],
       [6339, 8354]])

Random Classifier Xtrain and Xtest before data was upsampled and one hot encoded:
<br>0.6172258292592496
<br>0.6148991205380238

Therefore further cleaning and undersampling had a decrease effect on the Xtrain and Xtest

In [77]:
clf1 = RandomForestClassifier(max_depth=3,
                             min_samples_split=20,
                             min_samples_leaf =20)
cross_val_scores = cross_val_score(clf1, Xtrain_transformed, y_train, cv=10)
print(cross_val_scores.mean())

0.5934107228110783


In [118]:
import pickle
pickle.dump(clf,open('clf.p', 'wb'))